<a href="https://colab.research.google.com/github/dogukartal/ML-RoadMap/blob/main/RL/Hugging%20Face/PandaReachDense_v3/A2C/A2C_PandaReachDense_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay
!pip install stable-baselines3[extra]
!pip install gymnasium
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [3]:
import os
import gymnasium as gym
import panda_gym
from huggingface_sb3 import load_from_hub, package_to_hub
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from huggingface_hub import notebook_login

env_id = "PandaReachDense-v3"

env = gym.make(env_id)
env = make_vec_env(env_id, n_envs=4)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

s_size = env.observation_space.shape
a_size = env.action_space

model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)

model.learn(1_000_000)

model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

eval_env.render_mode = "rgb_array"
eval_env.training = False
eval_env.norm_reward = False

model = A2C.load("a2c-PandaReachDense-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

Streaming output truncated to the last 5000 lines.
|    value_loss         | 0.000148 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.76     |
|    ep_rew_mean        | -0.212   |
|    success_rate       | 1        |
| time/                 |          |
|    fps                | 412      |
|    iterations         | 23800    |
|    time_elapsed       | 1153     |
|    total_timesteps    | 476000   |
| train/                |          |
|    entropy_loss       | -1.07    |
|    explained_variance | 0.989    |
|    learning_rate      | 0.0007   |
|    n_updates          | 23799    |
|    policy_loss        | 0.00349  |
|    std                | 0.352    |
|    value_loss         | 9.58e-05 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.73     |
|    ep_rew_mean        | -0.216   |
|    success_rate       

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
